In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Data Cleaning:

In [ ]:
options = pd.read_csv('options_1h.csv')
options.head(10)

In [ ]:
options['Mid Open'] = (options['Open Bid'] + options['Open Ask'])/2
options['Mid Close'] = (options['Close Bid'] + options['Close Ask'])/2
options = options[options['Open'].isna() == False].reset_index(drop=True)

In [ ]:
spot = []
for i in range(options.shape[0]):
    if options['#RIC'][i] == 'SPY':
        val = options['Last'][i]
    
    spot.append(val)

options['Spot'] = spot
options = options[options['#RIC'] != 'SPY'].reset_index(drop=True)

In [ ]:
# Credit: Harshil Cherukuri

ORD_UPPERCASE_ADJ_CALLS = 64
ORD_UPPERCASE_ADJ_PUTS = 76
OPRA_NONROOT_LEN = 10

# return root,strike,expiry,type - eg. MSFT,157.70,2020-01-17,call
def parse_opra_ric(opra_ric: str):
    # skip null values - but, why are any of these null?
    if pd.isnull(opra_ric):
        return (None,None,None,None)

    try:
        ric, exchange_code = tuple(opra_ric.split("."))
    except:
        print(opra_ric)
        return(None,None,None,None)

    if exchange_code not in ["U"]:
        return (None,None,None,None)

    root_len = len(ric) - OPRA_NONROOT_LEN
    #root_len = len(ric)
    root = ric[:root_len]

    expiry_day = ric[-9:-7]
    raw_expiry_year = ric[-7:-5]
    if int(raw_expiry_year) <= 72:
        expiry_year = "20"+raw_expiry_year
    else:
        expiry_year = "19"+raw_expiry_year
    
    raw_expiry_month_put_or_call = ric[-10:-9]
    strike_ge_1000 = raw_expiry_month_put_or_call.islower()

    if ord(raw_expiry_month_put_or_call.upper()) <= (12 + ORD_UPPERCASE_ADJ_CALLS):
        contract_type = "call"
        expiry_month = str(ord(raw_expiry_month_put_or_call.upper()) - ORD_UPPERCASE_ADJ_CALLS).zfill(2)
    else:
        contract_type = "put"
        expiry_month = str(ord(raw_expiry_month_put_or_call.upper()) - ORD_UPPERCASE_ADJ_PUTS).zfill(2)

    expiry = f"{expiry_year}-{expiry_month}-{expiry_day}"

    if strike_ge_1000:
        strike = float(ric[-5:]) / 10
    else:
        strike = float(ric[-5:]) / 100

    return [root, strike, expiry, contract_type]

In [ ]:
opt_attr = []
for i in range(options.shape[0]):
    vals = pd.Series(parse_opra_ric(options['#RIC'][i]))
    opt_attr.append(vals)

opt_attr = pd.concat(opt_attr, axis=1).transpose()
opt_attr.columns = ['Ticker', 'Strike', 'Expiry', 'Opt Type']
opt_attr.head()

In [ ]:
optionsData = pd.concat([options, opt_attr], axis=1)
optionsData['T'] = [(pd.to_datetime(optionsData['Expiry'][i]) - \
                     pd.to_datetime(optionsData['Date-Time'][i][0:10])).days/252 for i in range(optionsData.shape[0])]
optionsData.head(10)

# Finding Implied Vol:

In [ ]:
from scipy.stats import norm

In [ ]:
# Returns the Black-Scholes-Merton (European) option price
def PV_BlackScholes(S, K, vol, r, t, d, typeof):
    # typeof == 'call' or 'put'
    d1 = (np.log(S/K) + (r-d+(pow(vol, 2)/2))*t)/(vol*np.sqrt(t))
    d2 = d1 - vol*np.sqrt(t)
    if typeof == 'call':
        return S*np.exp(-d*t)*norm.cdf(d1) - K*np.exp(-r*t)*norm.cdf(d2)
    else:
        return K*np.exp(-r*t)*norm.cdf(-d2) - S*np.exp(-d*t)*norm.cdf(-d1)

In [ ]:
def newtonImpliedVol(index: int, x0: float, error=0.001):
    S = optionsData['Spot'][index]
    K = optionsData['Strike'][index]
    r = 0.03
    d = 0
    T = optionsData['T'][index]
    typeof = optionsData['Opt Type'][index]
    true_price = optionsData['Last'][index]
    
    while PV_BlackScholes(S, K, x0, r, T, d, typeof) - true_price > error:
        f = PV_BlackScholes(S, K, x0, r, T, d, typeof) - true_price
        d1 = (np.log(S/K) + (r-d+pow(x0, 2)/2)*T)/(x0*np.sqrt(T))
        f_prime = S*np.sqrt(T)*norm.pdf(d1)
        x0 = x0 - (f/f_prime)
        
    return x0

# Selection Criteria:

In [ ]:
optTrain = pd.read_csv('df_train_no_nan.csv')  # Credit: Harshil Cherukuri
optTrain.index = optTrain['Date-Time']
optTrain = optTrain.drop('Date-Time', axis='columns')
optTrain.head()

In [ ]:
# Filtering the optionsData df beforehand so that it doesn't have to calculate IV 300,000+ times

indices = np.where(optionsData['#RIC'] == optTrain.columns[0])[0]
for i in range(1, len(optTrain.columns)):
    indices = np.concatenate([indices, np.where(optionsData['#RIC'] == optTrain.columns[i])[0]])

indices

In [ ]:
optionsData = optionsData.iloc[indices, :].reset_index(drop=True)
optionsData = optionsData.reset_index(drop=True)

In [ ]:
impVol = [newtonImpliedVol(i, 0.75) for i in range(optionsData.shape[0])]
optionsData['IV'] = impVol

In [ ]:
# Implied volatility time series
iv_ts = optionsData.pivot(index='Date-Time', columns='#RIC', values='IV')
iv_ts

# Trading Strategies:

## Basic Implied Volatility Strategy:

For this strategy, at each time step, we are longing the x options with the highest implied vol and shorting the x with the lowest implied vol.

In [ ]:
top = 3
strategyRet = []
for i in range(optTrain.shape[0]):
    colIndices = [*np.argsort(iv_ts.iloc[i, :])[0:top]] + [*np.argsort(iv_ts.iloc[i, :])[-top:]]
    longShort = np.array(top*[1] + top*[-1]).T
    ret = optTrain.iloc[i, colIndices]
    strategyRet.append(longShort @ ret)

In [ ]:
ivRet = np.array(strategyRet)
# plt.plot(np.cumprod(1+strategyRet)-1)
plt.plot(np.cumsum(ivRet))
plt.xlabel("Time")
plt.ylabel("Return")
plt.title("Long-Short Implied Vol Strategy")
plt.show()

## Dynamic Optimization:

In [ ]:
from cvxopt import matrix, solvers
solvers.options["show_progress"] = False

In [ ]:
def dynamicPortfolio(returns, window: int, top: int, min_w=0.01, max_w=0.4, balance=1):
        """
        Arguments:
        returns = Data frame or array of time series returns
        window = Sliding window which we use to slice the returns array to calculate the covariance matrix
        top = Selection criterion at each time step. We select the highest and lowest (top) options by implied vol
        min_w = Minimum weight for any particular option
        max_w = Maximum weight for any particular option
        balance = Portfolio rebalancing frequency
        """
        rows = returns.shape[0]
        cols = 2*top
        strategyRet = []
        for i in range(rows-window):
            if i % balance == 0:
                colIndices = [*np.argsort(iv_ts.iloc[i, :])[0:top]] + [*np.argsort(iv_ts.iloc[i, :])[-top:]]
                subset = returns.iloc[i:(i+window), colIndices]
                mu = np.mean(subset, axis=0)
                cov = np.array(subset.cov())
                Q = matrix(cov)
                r = matrix(-mu)
                # r = matrix(0.0, (cols, 1))
                G = matrix(np.vstack( [-np.identity(cols), np.identity(cols)] ))
                h = matrix(np.concatenate( (-min_w*np.ones(cols), max_w*np.ones(cols)) ))
                A = matrix(1.0, (1, cols))
                b = matrix(1.0)
                w = solvers.qp(Q, r, G, h, A, b)['x']
                ret = np.array(subset.iloc[0, :]).T @ np.array([*w])
                strategyRet.append(ret)
        
        return strategyRet

In [ ]:
dpRet = dynamicPortfolio(returns=optTrain, window=30, top=3)
plt.plot(np.cumsum(dpRet))
plt.xlabel("Time")
plt.ylabel("Return")
plt.title("Dynamic Portfolio")
plt.show()

## With IV Constraint:

In [ ]:
def ivConstrainedPortfolio(returns, window: int, top: int, max_iv: float, min_w=0.01, max_w=0.4, balance=1):
        rows = returns.shape[0]
        cols = 2*top
        strategyRet = []
        for i in range(rows-window):
            if i % balance == 0:
                colIndices = [*np.argsort(iv_ts.iloc[i, :])[0:top]] + [*np.argsort(iv_ts.iloc[i, :])[-top:]]
                subset = returns.iloc[i:(i+window), colIndices]
                mu = np.mean(subset, axis=0)
                cov = np.array(subset.cov())
                Q = matrix(cov)
                r = matrix(-mu)
                # r = matrix(0.0, (cols, 1))
                G = matrix(np.vstack( [np.diag(iv_ts.iloc[i, colIndices]), -np.identity(cols), np.identity(cols)] ))
                h = matrix(np.concatenate( (max_iv*np.ones(cols), -min_w*np.ones(cols), max_w*np.ones(cols)) ))
                A = matrix(1.0, (1, cols))
                b = matrix(1.0)
                w = solvers.qp(Q, r, G, h, A, b)['x']
                ret = np.array(subset.iloc[0, :]).T @ np.array([*w])
                strategyRet.append(ret)
        
        return strategyRet

In [ ]:
ivcRet = ivConstrainedPortfolio(returns=optTrain, window=30, top=3, max_iv=0.05)
plt.plot(np.cumsum(dpRet))
plt.plot(np.cumsum(ivcRet))
plt.xlabel("Time")
plt.ylabel("Return")
plt.legend(["Standard", "With IV Constraint"])
plt.title("Dynamic Portfolio")
plt.show()

In [ ]:
cumul_ret = [np.cumsum(ivRet)[-1], np.cumsum(dpRet)[-1], np.cumsum(ivcRet)[-1]]
avg_returns = np.array([np.mean(ivRet), np.mean(dpRet), np.mean(ivcRet)])
vols = np.array([np.std(ivRet), np.std(dpRet), np.std(ivcRet)])
sharpe = avg_returns/vols
performanceSummary = pd.DataFrame({'Total Return': cumul_ret, 'Avg Period Return': avg_returns, 'Period Volatility': vols, 
                      'Sharpe Ratio': sharpe}, index=['Long-Short IV Strategy', 'Dynamic Portfolio', 'IV-Constrained'])
performanceSummary

# Less Liquid Options and Testing Data:

In [ ]:
# Credit for CSV data: Harshil Cherukuri
lessLiquid = pd.read_csv('df_train_with_nan.csv')
lessLiquid.index = lessLiquid['Date-Time']
lessLiquid = lessLiquid.drop('Date-Time', axis='columns')

optTest = pd.read_csv('df_test.csv')
optTest.index = optTest['Date-Time']
optTest = optTest.drop('Date-Time', axis='columns')

lessLiquid.head()

In [ ]:
def fillna_avg(column):
    is_na = column.isna()
    if is_na[0]:
        column[0] = 0
    
    if is_na[is_na.shape[0]-1]:
        column[is_na.shape[0]-1] = 0
    
    for i in range(1, len(is_na)-1):
        if is_na[i]:
            column[i] = (column[i-1]+column[i+1])/2
    
    return column

In [ ]:
lessLiquid = lessLiquid.apply(fillna_avg, axis=0)
optTest = optTest.iloc[:-3, :].apply(fillna_avg, axis=0)

## Basic IV:

In [ ]:
strategyRet = []
for i in range(lessLiquid.shape[0]):
    colIndices = [*np.argsort(iv_ts.iloc[i, :])[0:top]] + [*np.argsort(iv_ts.iloc[i, :])[-top:]]
    longShort = np.array(top*[1] + top*[-1]).T
    ret = lessLiquid.iloc[i, colIndices]
    strategyRet.append(longShort @ ret)

In [ ]:
testStrategyRet = []
for i in range(optTest.shape[0]):
    colIndices = [*np.argsort(iv_ts.iloc[i, :])[0:top]] + [*np.argsort(iv_ts.iloc[i, :])[-top:]]
    longShort = np.array(top*[1] + top*[-1]).T
    ret = optTest.iloc[i, colIndices]
    testStrategyRet.append(longShort @ ret)

In [ ]:
ivRet = np.array(strategyRet)
plt.plot(np.cumsum(ivRet))
plt.xlabel("Time")
plt.ylabel("Return")
plt.title("Long-Short Implied Vol Strategy (Train)")
plt.show()

plt.plot(np.cumsum(testStrategyRet))
plt.xlabel("Time")
plt.ylabel("Return")
plt.title("Long-Short Implied Vol Strategy (Test)")
plt.show()

## Dynamic Portfolio:

In [ ]:
dpRet = dynamicPortfolio(returns=lessLiquid, window=30, top=3)
plt.plot(np.cumsum(dpRet))
plt.xlabel("Time")
plt.ylabel("Return")
plt.title("Dynamic Portfolio (Train)")
plt.show()

testDPRet = dynamicPortfolio(returns=optTest, window=30, top=3)
plt.plot(np.cumsum(testDPRet))
plt.xlabel("Time")
plt.ylabel("Return")
plt.title("Dynamic Portfolio (Test)")
plt.show()

## IV-Constrained Portfolio:

In [ ]:
ivcRet = ivConstrainedPortfolio(returns=lessLiquid, window=30, top=3, max_iv=0.05)
plt.plot(np.cumsum(dpRet))
plt.plot(np.cumsum(ivcRet))
plt.xlabel("Time")
plt.ylabel("Return")
plt.legend(["Standard", "With IV Constraint"])
plt.title("Dynamic Portfolio (Train)")
plt.show()

testIVCRet = ivConstrainedPortfolio(returns=optTest, window=30, top=3, max_iv=0.05)
plt.plot(np.cumsum(testDPRet))
plt.plot(np.cumsum(testIVCRet))
plt.xlabel("Time")
plt.ylabel("Return")
plt.legend(["Standard", "With IV Constraint"])
plt.title("Dynamic Portfolio (Test)")
plt.show()

## Second Train Data Performance:

In [ ]:
cumul_ret = [np.cumsum(ivRet)[-1], np.cumsum(dpRet)[-1], np.cumsum(ivcRet)[-1]]
avg_returns = np.array([np.mean(ivRet), np.mean(dpRet), np.mean(ivcRet)])
vols = np.array([np.std(ivRet), np.std(dpRet), np.std(ivcRet)])
sharpe = avg_returns/vols
performanceSummary = pd.DataFrame({'Total Return': cumul_ret, 'Avg Period Return': avg_returns, 'Period Volatility': vols, 
                      'Sharpe Ratio': sharpe}, index=['Long-Short IV Strategy', 'Dynamic Portfolio', 'IV-Constrained'])
performanceSummary

## Testing Data Performance:

In [ ]:
cumul_ret = [np.cumsum(testStrategyRet)[-1], np.cumsum(testDPRet)[-1], np.cumsum(testIVCRet)[-1]]
avg_returns = np.array([np.mean(testStrategyRet), np.mean(testDPRet), np.mean(testIVCRet)])
vols = np.array([np.std(testStrategyRet), np.std(testDPRet), np.std(testIVCRet)])
sharpe = avg_returns/vols
performanceSummary = pd.DataFrame({'Total Return': cumul_ret, 'Avg Period Return': avg_returns, 'Period Volatility': vols, 
                      'Sharpe Ratio': sharpe}, index=['Long-Short IV Strategy', 'Dynamic Portfolio', 'IV-Constrained'])
performanceSummary